In [8]:
import pandas as pd
import numpy as np
import datetime
import requests
import matplotlib.pyplot as plt

import geopandas as gpd

import zapimoveis_scraper as zap

from bs4 import BeautifulSoup

import folium
from folium.plugins import MarkerCluster

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [2]:
num = 50
casas = zap.search(localization='sp+ubatuba/',num_pages=num,acao='venda')

In [3]:
df = pd.DataFrame(index=range(0,len(casas)),columns=['endereco','valor','dorm','banheiro','vagas','area_tot','desc'])

In [4]:
for i in range(0,len(casas)):    
   
    #valor    
    df.iloc[i]['valor'] = casas[i].price
    #endereço
    df.iloc[i]['endereco'] = casas[i].address
    #dormitorios
    df.iloc[i]['dorm'] = casas[i].bedrooms
    #banheiros
    df.iloc[i]['banheiro'] = casas[i].bathrooms
    #vagas
    df.iloc[i]['vagas'] = casas[i].vacancies
    #área total
    df.iloc[i]['area_tot'] = casas[i].total_area_m2
    #descrição
    df.iloc[i]['desc'] = casas[i].description

df['data'] = pd.to_datetime(datetime.date.today())
df.to_excel('Zap_Imoveis_pages{}_{}.xlsx'.format(num,datetime.date.today()))

In [2]:
df = pd.read_excel('Zap_Imoveis_pages50_2021-11-08.xlsx')

In [4]:
df['loc'] = df["endereco"].apply(geolocator.geocode)

In [5]:
df["point"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

In [6]:
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

In [7]:
# Create a map object and center it to the avarage coordinates to m
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=2)
# if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster, add to m
marker_cluster = MarkerCluster().add_to(m)
# draw the markers and assign popup and hover texts
# add the markers the the cluster layers so that they are automatically clustered
for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    try:
        folium.Marker(location=location,
                          popup = r['valor'],
                          tooltip=(r['valor'],r['loc'][0].split(',')[0:2])).add_to(marker_cluster)
    except:
        pass
# display the map
m

In [10]:
df.to_excel('Zap_Imoveis_pages_lastversion_geoloc.xlsx')